# Stack Overflow Webscraping Project

Description here

In [95]:
# Import libraries and modules
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup

## Webscraping Stack Overflow

In [96]:
# Created an empty dataframe with the desired columns
df = pd.DataFrame(
    columns = [["company", "industry", "size", "year_founded", "status", "follower_count", "tech_stack", "benefits"]]
)

display(df)

,company,industry,size,year_founded,status,follower_count,tech_stack,benefits


In [97]:
# Created a function that collects spcific data points from stack overflow
def add_data_to_df(soupy):
    company_name = soupy.find("h1").text.strip() # Extract --> Company Name

    
    about_company = soupy.find_all("span", class_ = "d-block")


    indices = [12,13,14,15,16] # The indicies 12,13,14,15,and 16 represents industy, size, year founded, status, and followers
    values = []

    for index in indices: # This loop ignores IndexErrors when running the function for certain companies that are missing data
        try:
            values.append(about_company[index].text.strip())
        except IndexError:
            values.append(None) 

    industry = values[0]
    size = values[1]
    year_founded = values[2]
    status = values[3]
    followers = values[4]

    tech_skills = soupy.find_all("a", class_ = "flex--item s-tag no-tag-menu") # Searched for the company's tech stack
    tech_stack = [] # Created a list containing all relevant tech skills 

    for item in range(len(tech_skills)):
        tech_stack.append(tech_skills[item].text.strip())


    benefits_list = soupy.find_all("div", class_="flex--item pl8 pt2 fw-normal fs-body2 fc-black-700") # Searched for company benefits
    benefits = [] # Created a list containing all listed company benefits

    for item in range(len(benefits_list)):
        benefits.append(benefits_list[item].text.strip())
    
    
    # Appended all webscrapped elements into a list "new_row"

    new_row = [] 

    new_row.append(company_name)
    new_row.append(industry)
    new_row.append(size)
    new_row.append(year_founded)
    new_row.append(status)
    new_row.append(followers)
    new_row.append(tech_stack)
    new_row.append(benefits)

    df.loc[len(df.index)] = new_row # Added the list as a new row in the dataframe

In [98]:
# Accessing each link 

def scrape_page(page_link):

    company_list = page_link.find_all("a", class_="s-link", href=lambda href: href and "/jobs/companies" in href) # Accessed each <a> tag that includes href and "/jobs/companies"

    company_links = []

    for names in range(len(company_list)):
        company_links.append("https://stackoverflow.com" + company_list[names]["href"]) # The extracted links combines with "https://stackoverflow.com" generates the full link for each company's page on stack overflow


    # Created a for loop that loops through each company page at least three times
    for link in range(len(company_links)):  
        response = requests.get(company_links[link])
        max_attempts = 3 

        while max_attempts > 0:

            if response.status_code == 200:
                print("Successful connection")
                soupy = BeautifulSoup(response.text, "html")
                add_data_to_df(soupy)
                break

            elif response.status_code == 429:
                print("Response 429, reattempting...")
                time.sleep(5) # If the server is handling too many request --> wait five seconds and reattempt connection
                max_attempts -=1

            elif response.status_code == 403:
                print("Access Denied")
                break
            
            else:
                print(response.status_code)
                break

In [99]:
# Created a for loop that checks the connection of each page and scrapes data from each company's profile page through accessing their individual links

connection_attempts = 3
max_pages = 7

for page in range(1, max_pages + 1):
    url = f"https://stackoverflow.com/jobs/companies?pg={page}"
     
    while connection_attempts > 0:
        response = requests.get(url)

        if response.status_code == 200: # used a similar for loop structure to the "scrape_page" function created above
            print(f"Successfully connected to page:{page}")
            soup = BeautifulSoup(response.text, "html")
            scrape_page(soup)
            break

        elif response.status_code == 429:
            print("Response [429]: Reattempting to connect")
            time.sleep(5)
            connection_attempts -= 1

        elif response.status_code == 403:
                print("Response [403]: Access denied") 
                break
        else:
            print("Invalid link")
            break
    if connection_attempts == 0:
        print("Max number of attempts excceded")

Successfully connected to page:1
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successfully connected to page:2
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successfully connected to page:3
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successfully connected to page:4
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful

In [100]:
# Show the extracted data
display(df)

,company,industry,size,year_founded,status,follower_count,tech_stack,benefits
0,Pluralsight,SaaS,1k-5k employees,2004,VC Funded,399,"[.net, c#, c++, golang, java, javascript, kotl...","[Unlimited paid time off, Summer Fridays, Comp..."
1,ALDI SÜD IT,"E-Commerce, Information Technology, Retail",1k-5k employees,1913,Private,51,"[angular, asp.net-core, azure, bash, c#, conti...","[30 days of annual leave, Holiday and Christma..."
2,Nord Security,"Cybersecurity, Network Security, Software Deve...",1k-5k employees,2012,Private,482,"[php, go, mysql, apache-kafka, symfony, echo, ...",[The opportunity to shape a more trusted and p...
3,Volvo Group,Transportation,10k+ employees,1927,Private,378,"[reactjs, javascript, java, python, c#, amazon...","[Freedom to explore, try and create, Dresscode..."
4,EMBL-EBI (EMBL's European Bioinformatics Insti...,"Big Data, Data Science, Life Sciences",501-1k employees,1992,Public,164,"[javascript, java, angular, reactjs, node.js, ...","[Private Health Insurance, 30 days annual leav..."
...,...,...,...,...,...,...,...,...
57,UBS,"Financial Services, Financial Technology",10k+ employees,Private,103,None,"[javascript, java, kotlin, kubernetes, cicd, p...",[Working with cutting edge tech and agile prac...
58,ZEISS Group,"Machine Learning, Medical Devices, Virtual Rea...",10k+ employees,1846,Private,219,"[angular, swift, c#, azure, javascript, python...","[Flexible working time models, Open, dialogue-..."
59,Citi,"Finance, Financial Technology, Risk Management",10k+ employees,Public,192,None,"[java, python, api, management, c#, ai, cloud-...","[Medical, Dental and Vision coverage, Paid Tim..."
60,Capital One – Mexico City,"Financial Services, Financial Technology",10k+ employees,1994,Public,1,"[javascript, scala, python, java, cassandra, h...","[Health Insurance + Wellness, Vacation + Paid ..."


## Data Cleaning and Feature Engineering

### Correcting Misplaced Values

In [102]:
# Created a csv file based on the extracted data
df.to_csv("companies.csv", index = False)
df = pd.read_csv("companies.csv")
display(df)

,company,industry,size,year_founded,status,follower_count,tech_stack,benefits
0,Pluralsight,SaaS,1k-5k employees,2004,VC Funded,399,"['.net', 'c#', 'c++', 'golang', 'java', 'javas...","['Unlimited paid time off', 'Summer Fridays', ..."
1,ALDI SÜD IT,"E-Commerce, Information Technology, Retail",1k-5k employees,1913,Private,51,"['angular', 'asp.net-core', 'azure', 'bash', '...","['30 days of annual leave', 'Holiday and Chris..."
2,Nord Security,"Cybersecurity, Network Security, Software Deve...",1k-5k employees,2012,Private,482,"['php', 'go', 'mysql', 'apache-kafka', 'symfon...",['The opportunity to shape a more trusted and ...
3,Volvo Group,Transportation,10k+ employees,1927,Private,378,"['reactjs', 'javascript', 'java', 'python', 'c...","['Freedom to explore, try and create', 'Dressc..."
4,EMBL-EBI (EMBL's European Bioinformatics Insti...,"Big Data, Data Science, Life Sciences",501-1k employees,1992,Public,164,"['javascript', 'java', 'angular', 'reactjs', '...","['Private Health Insurance', '30 days annual l..."
...,...,...,...,...,...,...,...,...
57,UBS,"Financial Services, Financial Technology",10k+ employees,Private,103,NaN,"['javascript', 'java', 'kotlin', 'kubernetes',...",['Working with cutting edge tech and agile pra...
58,ZEISS Group,"Machine Learning, Medical Devices, Virtual Rea...",10k+ employees,1846,Private,219,"['angular', 'swift', 'c#', 'azure', 'javascrip...","['Flexible working time models', 'Open, dialog..."
59,Citi,"Finance, Financial Technology, Risk Management",10k+ employees,Public,192,NaN,"['java', 'python', 'api', 'management', 'c#', ...","['Medical, Dental and Vision coverage', 'Paid ..."
60,Capital One – Mexico City,"Financial Services, Financial Technology",10k+ employees,1994,Public,1,"['javascript', 'scala', 'python', 'java', 'cas...","['Health Insurance + Wellness', 'Vacation + Pa..."


#### Correcting Misplaced "industry" Column Values

In [103]:
# Identified the companies that have misplaced industry values
# Found that certain companies did not list their industry, causing the data to be shifted one value to the left
industry_list = df["industry"].values

for item in industry_list:
    if str("employees") in str(item): 
        display(df[df["industry"] == item])


,company,industry,size,year_founded,status,follower_count,tech_stack,benefits
49,Novo Nordisk A/S,10k+ employees,1923,Public,412,NaN,"['javascript', 'html', 'css', 'sql', 'python',...","['Working with highly skilled developers', 'Wo..."


,company,industry,size,year_founded,status,follower_count,tech_stack,benefits
55,KVK,1k-5k employees,1803,Public,53,NaN,"['.net', 'react', 'java', 'angular', 'c#', 'ty...","['Hybride werken', 'Agile werkwijze', 'Werken ..."


In [104]:
# Corrected the misplaced values by shifting values under the "industry", "size", "year_founded", "status", and "follower_count" one value to the right
# Removed the "NaN" value from the "follower_count" column and added it to the "industry" column
industry_list = df["industry"].values

for item in industry_list:
    if "employees" in item: 

        index = df[df["industry"] == item].index
        
        size = df.loc[index, "industry"]
        year_founded = df.loc[index, "size"]
        status = df.loc[index, "year_founded"]
        follower_count = df.loc[index, "status"]

        df.loc[index, "industry"] = None
        df.loc[index, "size"] = size
        df.loc[index, "year_founded"] = year_founded
        df.loc[index, "status"] = status
        df.loc[index, "follower_count"] = follower_count
        
        display(df.iloc[index])
    else: 
        pass


,company,industry,size,year_founded,status,follower_count,tech_stack,benefits
49,Novo Nordisk A/S,None,10k+ employees,1923,Public,412,"['javascript', 'html', 'css', 'sql', 'python',...","['Working with highly skilled developers', 'Wo..."


,company,industry,size,year_founded,status,follower_count,tech_stack,benefits
55,KVK,None,1k-5k employees,1803,Public,53,"['.net', 'react', 'java', 'angular', 'c#', 'ty...","['Hybride werken', 'Agile werkwijze', 'Werken ..."


#### Correcting Misplaced "year_founded" and "status" Column Values

In [105]:
# Identified companies that misplaced "year_founded" and "status" values 
# Noted that the rows with incorrect values for the "year_founded" column also have no value in its "follower_count" column, meaning these companies are missing "year_founded" data
year_founded_values = df["year_founded"].unique()

for value in year_founded_values:
    if ("Public" in value) or ("Private" in value) or ("VC Funded" in value):
        display(df[df["year_founded"] == value])
        

,company,industry,size,year_founded,status,follower_count,tech_stack,benefits
30,"Logius, onderdeel ministerie BZK","Computer Software, Government, IT Consulting",501-1k employees,Public,44,NaN,"['grafana', 'kibana', 'thanos', 'minio', 'harb...","['Individueel keuze budget (IKB) van 16,37% ov..."
43,Roblox,"Communications, Software Development, Virtual ...",1k-5k employees,Public,174,NaN,"['c#', 'c++', 'lua', 'python', 'node.js', 'mac...","['Robust medical, dental, and vision coverage'..."
45,Warner Bros. Discovery,"Broadcast, Digital Media, Entertainment",10k+ employees,Public,449,NaN,"['java', 'spring-boot', 'javascript', 'python-...","['Health insurance (medical, dental, vision, p..."
52,Caterpillar Inc.,"Internet of Things, Manufacturing, Software De...",10k+ employees,Public,438,NaN,"['java', 'javascript', 'python', 'scala', 'sql...","['Flexible and Hybrid Work options', 'Health, ..."
59,Citi,"Finance, Financial Technology, Risk Management",10k+ employees,Public,192,NaN,"['java', 'python', 'api', 'management', 'c#', ...","['Medical, Dental and Vision coverage', 'Paid ..."


,company,industry,size,year_founded,status,follower_count,tech_stack,benefits
31,Deutsche Börse Group,"Customer Data Platform, Financial Technology, ...",10k+ employees,Private,103,NaN,"['c++', 'java', 'python', 'google-cloud-platfo...",['HYBRID WORK – our working model combines the...
34,Gatekeeper,"Computer Software, Enterprise Software, Legal ...",51-200 employees,Private,42,NaN,"['ruby-on-rails', 'javascript', 'hotwire-rails...",['Fully remote working. Our team currently spa...
53,Smals,"Cloud Computing, Information Technology, Softw...",1k-5k employees,Private,127,NaN,"['java', 'javascript', '.net', 'php', 'c#', 's...",['Challenging projects with a positive impact ...
57,UBS,"Financial Services, Financial Technology",10k+ employees,Private,103,NaN,"['javascript', 'java', 'kotlin', 'kubernetes',...",['Working with cutting edge tech and agile pra...


In [106]:
# Shifted the "year_founded", "status", and "follower_count" column one cell to the right and set "year_founded" column values to None 
for value in year_founded_values:
    if ("Public" in value) or ("Private" in value) or ("VC Funded" in value):

        index = df[df["year_founded"] == value].index
        
        df.loc[index, "follower_count"] = df.loc[index, "status"] 
        df.loc[index, "status"] = df.loc[index, "year_founded"]
        df.loc[index, "year_founded"] = None

        display(df.iloc[index])
    else:
        pass
        

,company,industry,size,year_founded,status,follower_count,tech_stack,benefits
30,"Logius, onderdeel ministerie BZK","Computer Software, Government, IT Consulting",501-1k employees,None,Public,44,"['grafana', 'kibana', 'thanos', 'minio', 'harb...","['Individueel keuze budget (IKB) van 16,37% ov..."
43,Roblox,"Communications, Software Development, Virtual ...",1k-5k employees,None,Public,174,"['c#', 'c++', 'lua', 'python', 'node.js', 'mac...","['Robust medical, dental, and vision coverage'..."
45,Warner Bros. Discovery,"Broadcast, Digital Media, Entertainment",10k+ employees,None,Public,449,"['java', 'spring-boot', 'javascript', 'python-...","['Health insurance (medical, dental, vision, p..."
52,Caterpillar Inc.,"Internet of Things, Manufacturing, Software De...",10k+ employees,None,Public,438,"['java', 'javascript', 'python', 'scala', 'sql...","['Flexible and Hybrid Work options', 'Health, ..."
59,Citi,"Finance, Financial Technology, Risk Management",10k+ employees,None,Public,192,"['java', 'python', 'api', 'management', 'c#', ...","['Medical, Dental and Vision coverage', 'Paid ..."


,company,industry,size,year_founded,status,follower_count,tech_stack,benefits
31,Deutsche Börse Group,"Customer Data Platform, Financial Technology, ...",10k+ employees,None,Private,103,"['c++', 'java', 'python', 'google-cloud-platfo...",['HYBRID WORK – our working model combines the...
34,Gatekeeper,"Computer Software, Enterprise Software, Legal ...",51-200 employees,None,Private,42,"['ruby-on-rails', 'javascript', 'hotwire-rails...",['Fully remote working. Our team currently spa...
53,Smals,"Cloud Computing, Information Technology, Softw...",1k-5k employees,None,Private,127,"['java', 'javascript', '.net', 'php', 'c#', 's...",['Challenging projects with a positive impact ...
57,UBS,"Financial Services, Financial Technology",10k+ employees,None,Private,103,"['javascript', 'java', 'kotlin', 'kubernetes',...",['Working with cutting edge tech and agile pra...


### Spliting the "tech_stack" and "benefits" Columns into Multiple Columns

In [116]:
test_df = df.copy()
test_df

,company,industry,size,year_founded,status,follower_count,tech_stack,benefits
0,Pluralsight,SaaS,1k-5k employees,2004,VC Funded,399,"['.net', 'c#', 'c++', 'golang', 'java', 'javas...","['Unlimited paid time off', 'Summer Fridays', ..."
1,ALDI SÜD IT,"E-Commerce, Information Technology, Retail",1k-5k employees,1913,Private,51,"['angular', 'asp.net-core', 'azure', 'bash', '...","['30 days of annual leave', 'Holiday and Chris..."
2,Nord Security,"Cybersecurity, Network Security, Software Deve...",1k-5k employees,2012,Private,482,"['php', 'go', 'mysql', 'apache-kafka', 'symfon...",['The opportunity to shape a more trusted and ...
3,Volvo Group,Transportation,10k+ employees,1927,Private,378,"['reactjs', 'javascript', 'java', 'python', 'c...","['Freedom to explore, try and create', 'Dressc..."
4,EMBL-EBI (EMBL's European Bioinformatics Insti...,"Big Data, Data Science, Life Sciences",501-1k employees,1992,Public,164,"['javascript', 'java', 'angular', 'reactjs', '...","['Private Health Insurance', '30 days annual l..."
...,...,...,...,...,...,...,...,...
57,UBS,"Financial Services, Financial Technology",10k+ employees,None,Private,103,"['javascript', 'java', 'kotlin', 'kubernetes',...",['Working with cutting edge tech and agile pra...
58,ZEISS Group,"Machine Learning, Medical Devices, Virtual Rea...",10k+ employees,1846,Private,219,"['angular', 'swift', 'c#', 'azure', 'javascrip...","['Flexible working time models', 'Open, dialog..."
59,Citi,"Finance, Financial Technology, Risk Management",10k+ employees,None,Public,192,"['java', 'python', 'api', 'management', 'c#', ...","['Medical, Dental and Vision coverage', 'Paid ..."
60,Capital One – Mexico City,"Financial Services, Financial Technology",10k+ employees,1994,Public,1,"['javascript', 'scala', 'python', 'java', 'cas...","['Health Insurance + Wellness', 'Vacation + Pa..."


#### Spliting Up the "tech_stack" Column 

In [ ]:
# 
programing_url = "https://en.wikipedia.org/wiki/List_of_programming_languages"
response = requests.get(programing_url)
soup = BeautifulSoup(response.text, "html")


In [158]:
language_list = soup.find_all("a", href=lambda href: href and "/wiki/" in href)

for item in range(99, len(language_list)):
    x = language_list[item].text.strip()
    print(x)

A.NET (A#/A sharp)
A-0 System
A+ (A plus)
ABAP
ABC
ABC ALGOL
ACC
Accent (Rational Synergy)
Distributed Application Specification Language
Action!
ActionScript
Actor
Ada
ISO/IEC 8652
Adenine (Haystack)
AdvPL
Agda
Agilent VEE
Agora
AIMMS
Aldor
Alef
ALF
ALGOL 58
ALGOL 60
ALGOL 68
ALGOL W
Alice ML
Alma-0
AmbientTalk
Amiga E
AMPL
Analitik
AngelScript
Apache Pig latin
Apex (Salesforce.com, Inc)
APL
App Inventor for Android's visual block language (MIT App Inventor)
AppleScript
APT
Arc
ArkTS
ARexx
Argus
Assembly language (ASM)
AutoHotkey
AutoIt
AutoLISP / Visual LISP
Averest
AWK
Axum
B
Babbage
Ballerina
Bash
BASIC
Batch file (Windows/MS-DOS)
bc (basic calculator)
BCPL
BeanShell
BETA
BLISS
Blockly
BlooP
Boo
Boomerang
Bosque
C
C--
C++
C*
C#
C/AL
Caché ObjectScript
C Shell (csh)
Caml
Carbon
Catrobat
Cayenne (Lennart Augustsson)
Cecil
CESIL (Computer Education in Schools Instruction Language)
Céu
Ceylon
CFEngine
Cg (High-Level Shader/Shading Language [HLSL])
Ch
Chapel (Cascade High Productivity L

In [157]:
language_list[99:].text.strip()

AttributeError: 'list' object has no attribute 'text'

### Checking and Setting Column Data Types

In [107]:
# Checked the current data type for each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   company         62 non-null     object
 1   industry        60 non-null     object
 2   size            62 non-null     object
 3   year_founded    53 non-null     object
 4   status          62 non-null     object
 5   follower_count  61 non-null     object
 6   tech_stack      62 non-null     object
 7   benefits        62 non-null     object
dtypes: object(8)
memory usage: 4.0+ KB


In [108]:
df

,company,industry,size,year_founded,status,follower_count,tech_stack,benefits
0,Pluralsight,SaaS,1k-5k employees,2004,VC Funded,399,"['.net', 'c#', 'c++', 'golang', 'java', 'javas...","['Unlimited paid time off', 'Summer Fridays', ..."
1,ALDI SÜD IT,"E-Commerce, Information Technology, Retail",1k-5k employees,1913,Private,51,"['angular', 'asp.net-core', 'azure', 'bash', '...","['30 days of annual leave', 'Holiday and Chris..."
2,Nord Security,"Cybersecurity, Network Security, Software Deve...",1k-5k employees,2012,Private,482,"['php', 'go', 'mysql', 'apache-kafka', 'symfon...",['The opportunity to shape a more trusted and ...
3,Volvo Group,Transportation,10k+ employees,1927,Private,378,"['reactjs', 'javascript', 'java', 'python', 'c...","['Freedom to explore, try and create', 'Dressc..."
4,EMBL-EBI (EMBL's European Bioinformatics Insti...,"Big Data, Data Science, Life Sciences",501-1k employees,1992,Public,164,"['javascript', 'java', 'angular', 'reactjs', '...","['Private Health Insurance', '30 days annual l..."
...,...,...,...,...,...,...,...,...
57,UBS,"Financial Services, Financial Technology",10k+ employees,None,Private,103,"['javascript', 'java', 'kotlin', 'kubernetes',...",['Working with cutting edge tech and agile pra...
58,ZEISS Group,"Machine Learning, Medical Devices, Virtual Rea...",10k+ employees,1846,Private,219,"['angular', 'swift', 'c#', 'azure', 'javascrip...","['Flexible working time models', 'Open, dialog..."
59,Citi,"Finance, Financial Technology, Risk Management",10k+ employees,None,Public,192,"['java', 'python', 'api', 'management', 'c#', ...","['Medical, Dental and Vision coverage', 'Paid ..."
60,Capital One – Mexico City,"Financial Services, Financial Technology",10k+ employees,1994,Public,1,"['javascript', 'scala', 'python', 'java', 'cas...","['Health Insurance + Wellness', 'Vacation + Pa..."


In [115]:
# Set specific data types for each column in the dataset

"""
df["company"] = df["company"].astype("string")
df["industry"] = df["industry"].astype("string")
df["size"] = df["size"].astype("string")
df["year_founded"] = df["year_founded"].astype("int64") # issue with int and nonetypes
df["status"] = df["status"].astype("string")
df["follower_count"] = df["follower_count"].astype("int64")
"""


'\ndf["company"] = df["company"].astype("string")\ndf["industry"] = df["industry"].astype("string")\ndf["size"] = df["size"].astype("string")\ndf["year_founded"] = df["year_founded"].astype("int64") # issue with int and nonetypes\ndf["status"] = df["status"].astype("string")\ndf["follower_count"] = df["follower_count"].astype("int64")\n'

In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   company         62 non-null     object
 1   industry        60 non-null     object
 2   size            62 non-null     object
 3   year_founded    53 non-null     object
 4   status          62 non-null     object
 5   follower_count  61 non-null     object
 6   tech_stack      62 non-null     object
 7   benefits        62 non-null     object
dtypes: object(8)
memory usage: 4.0+ KB


In [111]:
# Separated the "tech_stack" column into multiple columns organized by type of tech
# New Columns: "Cloud Services and Infrastructure", "Programing Languages and Frameworks", "Databases and Data Technologies"
